In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## 测试一条两个样本的训练集

In [3]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
# 加标签,tokenizer出来的结果是一个dictionary，所以可以直接加入新的 key-value
batch['labels'] = torch.tensor([1, 1])
print(batch)  

optimizer = AdamW(model.parameters())
# 这里的 loss 是直接根据 batch 中提供的 labels 来计算的
# huggingface模型输出里有loss这一项
loss = model(**batch).loss  
loss.backward()
optimizer.step()

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([1, 1])}


## MRPC数据集-数据预处理

这里，我们使用MRPC数据集，它的全称是Microsoft Research Paraphrase Corpus，包含了5801个句子对，标签是两个句子是否是同一个意思。Huggingface有一个datasets库，可以让我们轻松地下载常见的数据集。

In [4]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

In [6]:
raw_train_dataset.features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [7]:
# 数据预处理
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
# 对于MRPC任务，我们不能把两个句子分开输入到模型中，二者应该组成一个pair输进去
# example
from pprint import pprint as print
inputs = tokenizer("first sentence", "second one")
print(inputs)
print(tokenizer.decode(inputs.input_ids))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 1148, 5650, 102, 1248, 1141, 102],
 'token_type_ids': [0, 0, 0, 0, 1, 1, 1]}
'[CLS] first sentence [SEP] second one [SEP]'


In [9]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

上述方法将所有数据读到内存中，又进行tokenizer，占用大量内存。
官方建议使用 **Dataset.map** 来调用一个分词方法，实现批量化的分词。<br>
**Dataset.map** Advantages：


*   The results of the function are cached, so it won't take any time if we re-execute the code.
*   It can apply multiprocessing to go faster than applying the function on each element of the dataset.
*  It does not load the whole dataset into memory, saving the results as soon as one element is processed.





In [10]:
def tokenize_function(sample):
    # 这里可以添加多种操作，不光是tokenize
    # 这个函数处理的对象，就是Dataset这种数据类型，通过features中的字段来选择要处理的数据
    return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

### Dynamic Padding
在划分batch的时候再进行padding，这样可以避免出现很多有一堆padding的序列，从而可以显著节省我们的训练时间。
需要用到**DataCollatorWithPadding**，来进行动态padding。

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
# 假设有一个size=5的batch
samples = tokenized_datasets['train'][:5]
samples.keys()

dict_keys(['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'])

In [16]:
# 去掉多余行
samples = {k:v for k,v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}  # 把这里多余的几列去掉
samples.keys()

dict_keys(['attention_mask', 'input_ids', 'label', 'token_type_ids'])

In [17]:
# 打印每个句子长度
[len(x) for x in samples["input_ids"]]

[52, 59, 47, 69, 60]

In [18]:
# 使用DataCollatorWithPadding处理
# samples中必须包含 input_ids 字段，因为这就是collator要处理的对象
batch = data_collator(samples)
[len(x) for x in batch['input_ids']]

[69, 69, 69, 69, 69]